# Data

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#import geocoders
#pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

print('Libraries imported.')

Libraries imported.


Light rail station locations and data from Foursquare, a website the collects venue information, will be used to provide the best rated destinations in proximity to the light rail to encourage people to take the light rail more. Light rail station locations come from the Sacramento Area Council of Governments which has a data repository available to the public. (http://data-sacog.opendata.arcgis.com/datasets/66dbe2e5407a443aa019f21839d4ebde_0) Information on venues within a half-mile (800 meters) radius from each light rail station will be collected by an explore-call to Foursquare’s API for each light rail location. Ratings for each venue will be collected by calls for each venue in the Foursquare API. 

Listings of the top rated venues, the top rated venues by light rail station, and maps will be created to promote top destinations for each light rail station.

There are over 1,000 venues within proximity of the light rail system. Receiving a venue rating is a premium call. There is a limit of 500 premium calls per day on the free version of Foursquare's API. Therefore, multiple ratings files will be created then appended. 

### Light Rail Station Location from SACOG

In [2]:
import pandas as pd
df_stations=pd.read_csv('/home/clif/Downloads/Light_Rail_Stops.csv')
df_stations.head()

,X,Y,OBJECTID,ID,POINT_X,POINT_Y,STATION,DISPLAY
0,6.725960e+06,1.988584e+06,453,0,-121.424940,38.621240,Marconi A,Marconi
1,6.749056e+06,1.969280e+06,454,0,-121.344592,38.567804,Butterfie,Butterfield
2,6.706857e+06,1.979433e+06,455,2,-121.491984,38.596426,Richds IB,Township 9
3,6.706847e+06,1.979402e+06,456,3,-121.492019,38.596340,Richds OB,Township 9
4,6.704692e+06,1.975047e+06,457,0,-121.499643,38.584416,SacVal IB,Sac Valley


Clean up stations dataframe

In [8]:
# drop unnecesary columns
df_stations.drop(['X','Y','OBJECTID', 'ID', 'STATION'],axis=1, inplace=True)
# Removed duplicate stations listed amd reset the index of new dataframe
df_stations.drop_duplicates(['DISPLAY'],keep='first',inplace=True)
df_stations.reset_index(drop=True, inplace=True)
# re-organize columns
df_stations=df_stations[['DISPLAY','POINT_Y','POINT_X']]
# rename columns
df_stations.rename(index=str, \
                   columns={'DISPLAY':'Station', 'POINT_X':'Longitude', 'POINT_Y' : 'Latitude'},\
                   inplace=True)

There are duplicate light rail stations. Identify and remove these stations/records

In [9]:
df_stations.head()

,Station,Latitude,Longitude
0,Marconi,38.621240,-121.424940
1,Butterfield,38.567804,-121.344592
2,Township 9,38.596426,-121.491984
3,Sac Valley,38.584416,-121.499643
4,7th & I,38.582659,-121.496810


Use Foursquare to bring in data on local venues.<br>
First define Foursquare credentials

In [10]:
# Four-Square Credentials
CLIENT_ID = 'JIVUEWG3G04BL5MZ1Y0FCOXQ5LEQDWNWZ0UEIVT5Q1VIHDI0' # your Foursquare ID
CLIENT_SECRET = 'PDOYIMEWSCZSZVZCYM2Y2ID4V4RHQWQ0YCCH1FAHZTSXP344' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create a function to get nearby venues from location

In [52]:
# Create a function to get nearby venues
LIMIT = 100 # limit number of venues returned by Foursquare API
radius = 800 # define radius to get all venues within one-half mile from zip code location
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
#            v['venue']['location']['address'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude',
                  'ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
#                  'Venue Address',
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
# retrieve venues near light rail stations from Foursquare
station_venues = getNearbyVenues(names=df_stations['Station'],
                                   latitudes=df_stations['Latitude'],
                                   longitudes=df_stations['Longitude']
                                  )

Marconi
Butterfield
Township 9
Sac Valley
7th & I
8th & K
Swanston
Historic Folsom
Iron Point
Glenn
Hazel
Sunrise
Cordova Town Center
Mather Field/Mills
Zinfandel
College Greens
Watt/Manlove
Starfire
Tiber
Broadway
4th Ave/Wayne Hultgren
City College
Fruitridge
47th Ave
Florin
Meadowview
Power Inn
University/65th St
59th Street
48th Street
39th Street
29th Street
23rd Street
16th Street
13th Street
8th & O
Archives Plaza
8th & Capitol
7th & Capitol
7th & K
9th & K
Cathedral Square
12th & I
Alkali Flats
Globe
Arden/Del Paso
Royal Oaks
Roseville Road
Watt/I-80
Morrison Creek - cnstrc
Frranklin Blvd
Center Parkway
Cosumbes River College
8th & H
8th 7 K


Venue Locations

In [54]:
station_venues.head()

,Station,Station Latitude,Station Longitude,ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marconi,38.62124,-121.42494,4d52bde4747f6dcbc25ed0d4,7-Eleven,38.620045,-121.419002,Convenience Store
1,Marconi,38.62124,-121.42494,4ea32b83003934222ebf8b47,"Ace Plumbing, Heating, & Air",38.621701,-121.426008,Construction & Landscaping
2,Marconi,38.62124,-121.42494,4c609665de6920a185649664,SACRT Light Rail Marconi/Arcade Station,38.621293,-121.424812,Light Rail Station
3,Marconi,38.62124,-121.42494,4c127cd5a1010f4797f14818,John Mackey Park,38.625028,-121.424829,Park
4,Marconi,38.62124,-121.42494,4e52b25418a86770c5882b90,Marconi Disc Golf 9 Hole Course,38.625279,-121.422965,Disc Golf


Venue Ratings

In [ ]:
# create dataframe with distinct venues 
print('station_venues dimensions', station_venues.shape)
unique_venues = station_venues['ID'].unique()
unique_venues = pd.DataFrame(data=unique_venues,columns=['ID'])
unique_venues.shape

In [ ]:
# generate venue ratings list
venue_ratings_list=[]
for x in range(0,500):
    try:
        venue_ratings_list.append({'ID': unique_venues['ID'][x], 
                                  'Rating': requests.get('https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'\
.format(unique_venues['ID'][x], CLIENT_ID, CLIENT_SECRET, VERSION)).json()['response']['venue']['rating']})
    except:
        venue_ratings_list.append({'ID': unique_venues['ID'][x], 'Rating': 'NaN'})

In [ ]:
# generate a dataframe from venue ratings list
Ratings_Final = pd.DataFrame(venue_ratings_list)

In [59]:
# append multiple ratings files
Ratings_Final = pd.read_csv('VenueRatings_0.csv')\
.append(pd.read_csv('VenueRatings_1.csv'),sort=True)
Ratings_Final = Ratings_Final.append(pd.read_csv('VenueRatings_2.csv'),sort=True)
Ratings_Final = Ratings_Final.append(pd.read_csv('VenueRatings_3.csv'),sort=True)
Ratings_Final = Ratings_Final.append(pd.read_csv('VenueRatings_missing.csv'),sort=True)\
.reset_index(drop=True)
Ratings_Final.head()

,ID,Rating,Venue
0,4d52bde4747f6dcbc25ed0d4,6.0,7-Eleven
1,4c609665de6920a185649664,NaN,SACRT Light Rail Marconi/Arcade Station
2,4f327c8619836c91c7dc823e,NaN,Ed's Fine Meats
3,4c127cd5a1010f4797f14818,NaN,John Mackey Park
4,5001770ff1366b23b4aa1bb9,NaN,"Hirst Tow Service, Inc"
